GANS PROJECT
For the Gans Project we need to work on getting information from avrious resources into our DB tables for further analysis
Starting with Getting information about flights for major cities in germany. Using the AeroDataBoxAPI we get the flights information.

In [ ]:
#Flights API
#Getting the arrivals for major cities in Germany form the AeroDataBoxAPI

import requests
from datetime import datetime
from datetime import timedelta
import pandas as pd
from IPython.display import JSON

#API key from the AeroDataBoxAPI
API_key = "API_KEY"

# Airports icoa
list_airport_icoa = ["EDDB","EDDF","EDDM","EDDK","EDDS"]


#company wants to know which flights will arrive the next day -Date Format.
#Add 1 day.
tomorrows_date = (datetime.now() + timedelta(days=1))
tomorrows_date_from=tomorrows_date.strftime("%Y-%m-%dT%H:%M")
#add 11 hours
future_date_and_time = tomorrows_date  + timedelta(hours=11)
tomorrows_date_to = future_date_and_time.strftime("%Y-%m-%dT%H:%M")
def get_flight_info(flight_json,i):
    # terminal
    try: terminal = flight_json['arrival']['terminal']
    except: terminal = None
    # aircraft
    try: aircraft = flight_json['aircraft']['model']
    except: aircraft = None

    return {
        'dep_airport':flight_json['departure']['airport']['name'],
        'sched_arr_loc_time':flight_json['arrival']['scheduledTimeLocal'],
        'terminal':terminal,
        'status':flight_json['status'],
        'aircraft':aircraft,
        'icao_code':i
    }
for i in list_airport_icoa:
    print(i)
    url = f"https://aerodatabox.p.rapidapi.com/flights/airports/icao/{i}/{tomorrows_date_from}/{tomorrows_date_to}"
    querystring = {"withLeg":"true","withCancelled":"true","withCodeshared":"true",
                   "withCargo":"true","withPrivate":"false","withLocation":"false"}

    headers = {
        "X-RapidAPI-Host": "aerodatabox.p.rapidapi.com",
        "X-RapidAPI-Key": API_key
    }
    response = requests.request("GET", url, headers=headers, params=querystring)
    arrivals = response.json()['arrivals']
    appended_data=pd.concat([appended_data, pd.DataFrame([get_flight_info(flight,i) for flight in arrivals])], ignore_index=True)

The API for openweathermap is used for getting the weather information

In [ ]:
#weather information
from pyowm import OWM
from pyowm.utils import config
from pyowm.utils import timestamps
import pandas as pd
import requests

OWM_key = 'Own OWM key'

city = ["Berlin", "Cologne", "Frankfurt", "Munich", "Stuttgart" ]
country = "DE"

forecast_api = []
for i in city:
    response = requests.get(f'http://api.openweathermap.org/data/2.5/forecast/?q={i},{country}&appid={OWM_key}&units=metric&lang=en')
    forecast_api.append(response.json())
 # datetime, temperature, wind, prob_perc, rain_qty, snow 
weather_info = []
for forecast_3h in forecast_api:
    weather_hour = {'municipality_iso_country': forecast_3h['city']['name'] + ',' + forecast_3h['city']['country']}
    for hour in forecast_3h['list']:
        # datetime utc
        weather_hour['datetime'] = hour['dt_txt']
        # temperature 
        weather_hour['temperature'] = hour['main']['temp']
        # wind
        weather_hour['wind'] = hour['wind']['speed']
        # probability precipitation 
        try: weather_hour['prob_perc'] = float(hour['pop'])
        except: weather_hour['prob_perc'] = 0
        # rain
        try: weather_hour['rain_qty'] = float(hour['rain']['3h'])
        except: weather_hour['rain_qty'] = 0
        # wind 
        try: weather_hour['snow'] = float(hour['snow']['3h'])
        except: weather_hour['snow'] = 0
        weather_info.append(weather_hour.copy())
        # weather_de = pd.concat(weather_de,pd.DataFrame.from_dict(weather_info), ignore_index=True)

weather_de = pd.DataFrame(weather_info)

Get the demographics information about the cities from Wikipedia

In [ ]:
#Using Beautiful Soup to scrape data from the internet(Wikipedia pages for infomation on the cities)

import bs4
from bs4 import BeautifulSoup as bs
import unicodedata
import requests
import pandas as pd
import re

cities = ['Berlin','Frankfurt','Stuttgart','Munich','Cologne']

def City_info(soup):
    
    ret_dict = {}
    ret_dict['city'] = soup.h1.get_text()
    
    
    if soup.select_one('.mergedrow:-soup-contains("Mayor")>.infobox-label') != None:
        i = soup.select_one('.mergedrow:-soup-contains("Mayor")>.infobox-label')
        mayor_name_html = i.find_next_sibling()
        mayor_name = unicodedata.normalize('NFKD',mayor_name_html.get_text())
        ret_dict['mayor']  = mayor_name
    
    if soup.select_one('.mergedrow:-soup-contains("City")>.infobox-label') != None:
        j =  soup.select_one('.mergedrow:-soup-contains("City")>.infobox-label')
        area = j.find_next_sibling('td').get_text()
        ret_dict['city_size'] = unicodedata.normalize('NFKD',area)

    if soup.select_one('.mergedtoprow:-soup-contains("Elevation")>.infobox-data') != None:
        k = soup.select_one('.mergedtoprow:-soup-contains("Elevation")>.infobox-data')
        elevation_html = k.get_text()
        ret_dict['elevation'] = unicodedata.normalize('NFKD',elevation_html)
    
    if soup.select_one('.mergedtoprow:-soup-contains("Population")') != None:
        l = soup.select_one('.mergedtoprow:-soup-contains("Population")')
        c_pop = l.findNext('td').get_text()
        ret_dict['city_population'] = c_pop
    
    if soup.select_one('.infobox-label>[title^=Urban]') != None:
        m = soup.select_one('.infobox-label>[title^=Urban]')
        u_pop = m.findNext('td')
        ret_dict['urban_population'] = u_pop.get_text()

    if soup.select_one('.infobox-label>[title^=Metro]') != None:
        n = soup.select_one('.infobox-label>[title^=Metro]')
        m_pop = n.findNext('td')
        ret_dict['metro_population'] = m_pop.get_text()
    
    if soup.select_one('.latitude') != None:
        o = soup.select_one('.latitude')
        ret_dict['lat'] = o.get_text()

    if soup.select_one('.longitude') != None:    
        p = soup.select_one('.longitude')
        ret_dict['long'] = p.get_text()
    
    return ret_dict

list_of_city_info = []
for city in cities:
    url = 'https://en.wikipedia.org/wiki/{}'.format(city)
    web = requests.get(url,'html.parser')
    soup = bs(web.content)
    list_of_city_info.append(City_info(soup))
df_cities = pd.DataFrame(list_of_city_info)

Reading the information about the airports from csv file

In [ ]:
#reading data from csv file for information on the airports of the cities
import pandas as pd

airports_cities = (
pd.read_csv('API-GANS/airports.csv')
    .query('type == "large_airport"')
    .filter(['name','latitude_deg','longitude_deg','iso_country','iso_region','municipality','gps_code','iata_code'])
    .rename(columns={'gps_code':'icao_code'})
    .assign(municipality_iso_country = lambda x: x['municipality'] + ',' + x['iso_country'])
)

Get information about the time of Sunrise or Sunset of a given location

In [62]:
# Information on Sun's position in city
import requests
from datetime import datetime
from datetime import timedelta
tomorrows_date = (datetime.now() + timedelta(days=1))
latitude ="6,9"
API_Key="API-Key"
#Cologne
url = "https://geo-services-by-mvpc-com.p.rapidapi.com/sun_positions"
querystring = {"location":latitude,"date":tomorrows_date}
headers = {
	"X-RapidAPI-Host": "geo-services-by-mvpc-com.p.rapidapi.com",
	"X-RapidAPI-Key":API_Key
}

response = requests.request("GET", url, headers=headers, params=querystring)
import pandas as pd
sun_data_c= pd.json_normalize(response.json()['data'])
# next step: select the columns you want to incude on your database
sun_data_c = (
sun_data_c
    .filter(['sunrise','sunset','dawn','dusk','night','goldenHour' ])
    .assign(city = 'Colonge,DE')
)
  

In [ ]:
import requests
from datetime import datetime
from datetime import timedelta
tomorrows_date = (datetime.now() + timedelta(days=1))
API_Key="API_key"
latitude ="9,1"
#Stuttgart
url = "https://geo-services-by-mvpc-com.p.rapidapi.com/sun_positions"
querystring = {"location":latitude,"date":tomorrows_date}
headers = {
	"X-RapidAPI-Host": "geo-services-by-mvpc-com.p.rapidapi.com",
	"X-RapidAPI-Key":API_Key
}

response = requests.request("GET", url, headers=headers, params=querystring)
import pandas as pd
sun_data_s= pd.json_normalize(response.json()['data'])
# next step: select the columns you want to incude on your database
sun_data_s= (
    sun_data_s
        .filter(['sunrise','sunset','dawn','dusk','night','goldenHour' ])
        .assign(city = 'Stuttgart,DE')
    )

In [ ]:
# Information on Sun's position in city
import requests
from datetime import datetime
from datetime import timedelta
tomorrows_date = (datetime.now() + timedelta(days=1))
API_Key="API_key"
latitude ="8,68"
#Frankfurt
url = "https://geo-services-by-mvpc-com.p.rapidapi.com/sun_positions"
querystring = {"location":latitude,"date":tomorrows_date}
headers = {
	"X-RapidAPI-Host": "geo-services-by-mvpc-com.p.rapidapi.com",
	"X-RapidAPI-Key":API_Key
}

response = requests.request("GET", url, headers=headers, params=querystring)
import pandas as pd
sun_data_f= pd.json_normalize(response.json()['data'])
# next step: select the columns you want to incude on your database
sun_data_f= (
sun_data_f
    .filter(['sunrise','sunset','dawn','dusk','night','goldenHour' ])
    .assign(city = 'Frankfurt,DE')
)

In [ ]:
# Information on Sun's position in city
import requests
from datetime import datetime
from datetime import timedelta
tomorrows_date = (datetime.now() + timedelta(days=1))
API_Key="API_key"
latitude ="13,4"
#Berlin
url = "https://geo-services-by-mvpc-com.p.rapidapi.com/sun_positions"
querystring = {"location":latitude,"date":tomorrows_date}
headers = {
	"X-RapidAPI-Host": "geo-services-by-mvpc-com.p.rapidapi.com",
	"X-RapidAPI-Key":API_Key
}

response = requests.request("GET", url, headers=headers, params=querystring)
import pandas as pd
sun_data_b= pd.json_normalize(response.json()['data'])
# next step: select the columns you want to incude on your database
sun_data_b = (
sun_data_b
    .filter(['sunrise','sunset','dawn','dusk','night','goldenHour' ])
    .assign(city = 'Berlin,DE')
)

In [ ]:
import requests
from datetime import datetime
from datetime import timedelta
tomorrows_date = (datetime.now() + timedelta(days=1))
API_Key="API_key"
latitude ="11,5"
#Munich
url = "https://geo-services-by-mvpc-com.p.rapidapi.com/sun_positions"
querystring = {"location":latitude,"date":tomorrows_date}
headers = {
	"X-RapidAPI-Host": "geo-services-by-mvpc-com.p.rapidapi.com",
	"X-RapidAPI-Key": APIKey
}

response = requests.request("GET", url, headers=headers, params=querystring)
import pandas as pd
sun_data_m= pd.json_normalize(response.json()['data'])
# next step: select the columns you want to incude on your database
sun_data_m= (
    sun_data_m
        .filter(['sunrise','sunset','dawn','dusk','night','goldenHour' ])
        .assign(city = 'Munich,DE')
    )

In [ ]:
DE_Sun = [sun_data_s, sun_data_f, sun_data_m, sun_data_b, sun_data_k ]
DE_Sundata = pd.concat(DE_Sun)

Insert the information into the local database 

In [ ]:
#Connecting to the local databse
import pandas as pd
import sqlalchemy
import pymysql
schema="name_of_schema"
host="host_name"
user="user_name"
password="user_password"
port=3306
con = f'mysql+pymysql://{user}:{password}@{host}:{port}/{schema}'

In [ ]:
#create cities table and on execution the second time append the data in the existing database
cities.dropna().to_sql('cities', con=con, if_exists='append', index=False)

In [ ]:
#create airports cities table and on execution the second time append the data in the existing database
airports_cities.dropna().to_sql('airports_cities', if_exists='append', con=con, index=False)

In [ ]:
#create weather table and on execution the second time append the data in the existing database
weather_result.assign(datetime = lambda x: pd.to_datetime(x['datetime'])).to_sql('weather', if_exists='append', con=con, index=False)

In [ ]:
#create arrivals table and on execution the second time append the data in the existing database
import numpy as np
(
arrivals_DE
    .replace({np.nan},'unknown')
    .assign(sched_arr_loc_time = lambda x: pd.to_datetime(x['sched_arr_loc_time']))
    .to_sql('arrivals', if_exists='append', con=con, index=False))

In [ ]:
#create sundata table and on execution the second time append the data in the existing database
DE_Sundata.dropna().to_sql('sundata', if_exists='append', con=con, index=False)

In [ ]:
#Connect to AWS database instance to insert data from the csv file
import pandas as pd
import sqlalchemy

schema="schema_name"
host="Endpoint from AWS"
user="user_name"
password="Password"
port=3306
con = f'mysql+pymysql://{user}:{password}@{host}:{port}/{schema}'

(
 airports_cities.dropna().to_sql('airports', if_exists='append', con=con, index=False)
)